In [1]:
import os
import pandas as pd
import praw
import pandas as pd
from datetime import datetime, timedelta, timezone
import os
import psaw as ps
from dotenv import load_dotenv
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F
import torch
from tqdm import tqdm
from psaw import PushshiftAPI
from praw.exceptions import APIException
from transformers import AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer


# 📌 Pfade zu den CSV-Dateien
DRIVE_PATH = "G:/Meine Ablage/reddit/"
POSTS_CSV = os.path.join(DRIVE_PATH, "reddit_posts.csv")
COMMENTS_CSV = os.path.join(DRIVE_PATH, "reddit_comments.csv")

# 🔹 **Funktion zum Laden der CSV-Dateien**
def load_csv(filepath):
    """Lädt eine CSV-Datei mit `|` als Trennzeichen und Debugging-Infos"""
    if not os.path.exists(filepath):
        print(f"❌ Datei nicht gefunden: {filepath}")
        return pd.DataFrame()

    df = pd.read_csv(filepath, sep="|", encoding="utf-8-sig", on_bad_lines="skip")

    print(f"\n📌 Datei geladen: {filepath}")
    print(f"🔹 Spalten: {df.columns.tolist()}")
    print(df.dtypes)
    print(df.head())

    return df

# 📌 **Daten laden**
df_posts = load_csv(POSTS_CSV)
df_comments = load_csv(COMMENTS_CSV)


c:\Users\hp\anaconda3\envs\crypto\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



📌 Datei geladen: G:/Meine Ablage/reddit/reddit_posts.csv
🔹 Spalten: ['post_id', 'crypto', 'search_term', 'subreddit', 'author', 'date', 'time', 'score', 'num_comments', 'full_text', 'sentiment', 'sentiment_confidence']
post_id                  object
crypto                   object
search_term              object
subreddit                object
author                   object
date                     object
time                     object
score                     int64
num_comments              int64
full_text                object
sentiment                object
sentiment_confidence    float64
dtype: object
   post_id   crypto search_term       subreddit               author  \
0  1j0dz4r  Bitcoin     bitcoin  CryptoCurrency             diwalost   
1  1j0dghm  Bitcoin     bitcoin  CryptoCurrency        3fkgf9fmd980e   
2  1j0cx0g  Bitcoin     bitcoin  CryptoCurrency            kirtash93   
3  1j0cnlh  Bitcoin     bitcoin  CryptoCurrency                KIG45   
4  1j08i0y  Bitcoin   

In [ ]:

df_comments.head()

,post_id,comment_id,author,date,time,score,full_text,sentiment,sentiment_confidence
0,1j0dz4r,mfb16ty,DonasAskan,2025-02-28,19:43:55,1,How does this only have 3 upvotes lol?,neutral,0.816071
1,1j0dz4r,mfakfdl,Deeujian,2025-02-28,18:26:05,-2,1-2% is nothing for Blackrock but they decided...,neutral,0.578827
2,1j0dz4r,mfag4zu,diwalost,2025-02-28,18:05:51,1,"They are not, retail is",neutral,0.691663
3,1j0dz4r,mfbrf7n,Bear-Bull-Pig,2025-02-28,21:46:12,1,People don't like Blackrock,neutral,0.493956
4,1j0dz4r,mfao0qa,diwalost,2025-02-28,18:42:51,1,They filled for ETF when we were in bear marke...,neutral,0.619266


In [13]:
df_posts.head()

,post_id,crypto,search_term,subreddit,author,date,time,score,num_comments,full_text,sentiment,sentiment_confidence
0,1j0dz4r,Bitcoin,bitcoin,CryptoCurrency,diwalost,2025-02-28,18:01:02,13,8,BlackRock Adds Its Bitcoin ETF to Model Portfo...,neutral,0.602603
1,1j0dghm,Bitcoin,bitcoin,CryptoCurrency,3fkgf9fmd980e,2025-02-28,17:39:22,0,1,"x-post: As the free-float of coins is low, is ...",bullish,0.520766
2,1j0cx0g,Bitcoin,bitcoin,CryptoCurrency,kirtash93,2025-02-28,17:16:50,63,33,Companies Building on the Bitcoin Lightning Ne...,bullish,0.669048
3,1j0cnlh,Bitcoin,bitcoin,CryptoCurrency,KIG45,2025-02-28,17:06:18,34,44,"Michael Saylor Says 'Sell A Kidney, But Keep T...",neutral,0.644649
4,1j08i0y,Bitcoin,bitcoin,CryptoCurrency,rizzobitcoinhistory,2025-02-28,14:09:15,426,23,Teaching Bitcoin on Wall Street at $30 in 2012...,bullish,0.803598


In [2]:
posts = df_posts.copy()
comments = df_comments.copy()

In [3]:
# 🔹 **Relevante Spalten für den Merge**
posts = posts[["post_id", "crypto", "search_term", "subreddit", "author", "date", "time", "score", "full_text", "sentiment", "sentiment_confidence"]]
comments = comments[["post_id", "comment_id", "author", "date", "time", "score", "full_text", "sentiment", "sentiment_confidence"]]

# 🔹 **Kommentare erben `crypto`, `search_term` und `subreddit` vom Post**
comments = comments.merge(df_posts[["post_id", "crypto", "search_term", "subreddit"]], on="post_id", how="left")

# 🔹 `type`-Spalte für Unterscheidung hinzufügen
posts["comment_id"] = None  # Posts haben keine comment_id
posts["type"] = "post"
comments["type"] = "comment"

# 🔹 **Gemeinsame Spalten für den Merge**
common_columns = [
    "post_id", "comment_id", "type", "date", "time", "crypto",
    "search_term", "subreddit", "author","full_text","score", "sentiment", "sentiment_confidence",
]

# 🔹 **Merging der Daten (Posts + Kommentare)**
df_merged = pd.concat([posts[common_columns], comments[common_columns]], ignore_index=True)

# 🔍 Debugging: Überprüfung der Größe
print(f"📌 Merged Dataset: {df_merged.shape[0]} Einträge (Posts + Kommentare)")

# 🔹 Überprüfen, ob alles korrekt normalisiert wurde
print(df_merged.head())

📌 Merged Dataset: 321321 Einträge (Posts + Kommentare)
   post_id comment_id  type        date      time   crypto search_term  \
0  1j0dz4r       None  post  2025-02-28  18:01:02  Bitcoin     bitcoin   
1  1j0dghm       None  post  2025-02-28  17:39:22  Bitcoin     bitcoin   
2  1j0cx0g       None  post  2025-02-28  17:16:50  Bitcoin     bitcoin   
3  1j0cnlh       None  post  2025-02-28  17:06:18  Bitcoin     bitcoin   
4  1j08i0y       None  post  2025-02-28  14:09:15  Bitcoin     bitcoin   

        subreddit               author  \
0  CryptoCurrency             diwalost   
1  CryptoCurrency        3fkgf9fmd980e   
2  CryptoCurrency            kirtash93   
3  CryptoCurrency                KIG45   
4  CryptoCurrency  rizzobitcoinhistory   

                                           full_text  score sentiment  \
0  BlackRock Adds Its Bitcoin ETF to Model Portfo...     13   neutral   
1  x-post: As the free-float of coins is low, is ...      0   bullish   
2  Companies Building on the

In [5]:
df_merged.head()

,post_id,comment_id,type,date,time,crypto,search_term,subreddit,author,full_text,score,sentiment,sentiment_confidence
0,1j0dz4r,None,post,2025-02-28,18:01:02,Bitcoin,bitcoin,CryptoCurrency,diwalost,BlackRock Adds Its Bitcoin ETF to Model Portfo...,13,neutral,0.602603
1,1j0dghm,None,post,2025-02-28,17:39:22,Bitcoin,bitcoin,CryptoCurrency,3fkgf9fmd980e,"x-post: As the free-float of coins is low, is ...",0,bullish,0.520766
2,1j0cx0g,None,post,2025-02-28,17:16:50,Bitcoin,bitcoin,CryptoCurrency,kirtash93,Companies Building on the Bitcoin Lightning Ne...,63,bullish,0.669048
3,1j0cnlh,None,post,2025-02-28,17:06:18,Bitcoin,bitcoin,CryptoCurrency,KIG45,"Michael Saylor Says 'Sell A Kidney, But Keep T...",34,neutral,0.644649
4,1j08i0y,None,post,2025-02-28,14:09:15,Bitcoin,bitcoin,CryptoCurrency,rizzobitcoinhistory,Teaching Bitcoin on Wall Street at $30 in 2012...,426,bullish,0.803598


In [6]:
top_coins = [
    "Bitcoin", "Ethereum", "Tether", "Ripple", "Binance Coin", "Solana", "USD Coin",
    "Dogecoin", "Cardano", "TRON", "Chainlink", "Avalanche", "Toncoin", "Stellar",
    "Hedera", "Shiba Inu", "Polkadot", "UNUS SED LEO", "Litecoin", "Bitcoin Cash",
    "NEAR", "MATIC", "XRP", "CRO", "Band", "Optimism", "Celestia", "Numerai",
    "Atheir", "Sui", "HZPE"
]


In [8]:
filtered_df = df_merged[df_merged["crypto"].isin(top_coins)]


In [9]:
# Setze den Pfad zu deinem Google Drive Ordner
DRIVE_PATH = "G:/Meine Ablage/reddit/"
APP_CSV = os.path.join(DRIVE_PATH, "app.csv")

In [10]:
def append_to_csv(df_new, filename, key_column):
    """Hängt neue Daten an eine bestehende CSV an & entfernt Duplikate."""
    file_path = os.path.join(DRIVE_PATH, filename)

    try:
        # Falls Datei existiert, alte Daten einlesen
        if os.path.exists(file_path):
            df_existing = pd.read_csv(file_path, sep="|", encoding="utf-8-sig", on_bad_lines="skip")
            
            # 🔹 Daten zusammenführen & Duplikate nach `key_column` entfernen (neuere Werte behalten)
            df_combined = pd.concat([df_existing, df_new], ignore_index=True).drop_duplicates(subset=[key_column], keep="last")
        else:
            df_combined = df_new  # Falls keine Datei existiert, neue Daten direkt nutzen

        # 🔹 CSV speichern
        df_combined.to_csv(
            file_path,
            index=False,
            sep="|",
            encoding="utf-8-sig",
            lineterminator="\n"
        )
        print(f"✅ Datei erfolgreich aktualisiert: {file_path}")

    except Exception as e:
        print(f"Fehler beim Speichern der Datei {filename}: {e}")

def export_to_drive(df_merged):
    """Speichert Posts, Kommentare & die gemergte Datei mit Duplikat-Prüfung."""
    try:
        append_to_csv(df_merged, "app.csv", key_column="post_id")
    except Exception as e:
        print(f"Fehler beim Export: {e}")

In [11]:
# Export-Funktion aufrufen
export_to_drive(df_merged)

✅ Datei erfolgreich aktualisiert: G:/Meine Ablage/reddit/app.csv
